* prbrown 20201012 17:35

In [ ]:
import numpy as np
import gdxpds
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, sys, math, site
from tqdm.notebook import tqdm, trange
from glob import glob

import geopandas as gpd
import shapely
os.environ['PROJ_NETWORK'] = 'OFF'

reedspath = os.path.dirname(os.getcwd())
remotepath = '/Volumes/ReEDS/' if sys.platform == 'darwin' else r'//nrelnas01/ReEDS/'

### Format plots and load other convenience functions
site.addsitedir(os.path.join(reedspath,'postprocessing'))
import plots
import reedsplots as rplots
plots.plotparams()

# Specify cases to compare

In [ ]:
runspath = os.path.join(remotepath,'Users/pbrown/ReEDSruns/20211005_transcost/')
casebase = 'v20211112_transvintG0_Z35'
casecomp = 'v20211112_transvintG0_Z35_VSCall'

# runspath = os.path.join(reedspath,'runs',)
# casebase = 'v20211112_transvintG0_Z35'
# casecomp = 'v20211112_transvintG0_Z35_VSCall'

# Plots from bokeh

In [ ]:
plotvals = [
    '3_Generation (TWh)',
    '4_Capacity (GW)',
    '5_New Annual Capacity (GW)',
    '6_Annual Retirements (GW)',
    '7_Final Gen by timeslice (GW)',
    '11_Firm Capacity (GW)',
    '12_Curtailment Rate',
    '14_Transmission (GW-mi)',
    '15_Bulk System Electricity Pric',
    '18_National Average Electricity',
    '24_2020-2050 Present Value of S',
]
for val in plotvals:
    plt.close()
    f,ax,leg,dfdiff = rplots.plotdiff(
        val, casebase, casecomp, runspath, titleshorten=10)
#     for col in range(3):
#         ax[col].set_xlim(2009,2051)
#     ax[2].set_ylim(-100,100)
    plt.show()

In [ ]:
onlytechs = ['wind-ons','wind-ofs','upv','dupv']
for val in ['3_Generation (TWh)','4_Capacity (GW)',
            # '5_New Annual Capacity (GW)','6_Annual Retirements (GW)',
           ]:
    f,ax,leg,dfdiff = rplots.plotdiff(
        val, casebase, casecomp, runspath, titleshorten=10, onlytechs=onlytechs)
    plt.show()

# Transmission maps

In [ ]:
### Get the BA map
dfba = gpd.read_file(os.path.join(reedspath,'inputs','shapefiles','US_PCA')).set_index('rb')
endpoints = (
    gpd.read_file(os.path.join(reedspath,'inputs','shapefiles','transmission_endpoints'))
    .set_index('ba_str'))
endpoints['x'] = endpoints.centroid.x
endpoints['y'] = endpoints.centroid.y
# dfba['x'] = dfba.geometry.centroid.x
# dfba['y'] = dfba.geometry.centroid.y
dfba['x'] = dfba.index.map(endpoints.x)
dfba['y'] = dfba.index.map(endpoints.y)
dfba.st = dfba.st.str.upper()

### Aggregate to states
dfstates = dfba.dissolve('st')

In [ ]:
year = 2036
wscale = 0.0002
rplots.plot_trans_diff(
    casebase=casebase, casecomp=casecomp, runspath=runspath, pcalabel=False, wscale=wscale,
    yearlabel=False, year=year,
)

In [ ]:
### Use straight lines
alpha = 1
year = 2036
routes, wscale = False, 0.0004
labelcolors = {'AC':'C2','DC':'C1','VSC':'C3'}
plt.close()
f,ax = rplots.plot_trans_onecase(
    case=os.path.join(runspath,casecomp), pcalabel=False,
    wscale=wscale, routes=routes,
    yearlabel=False, year=year, alpha=alpha,
    simpletypes=None,
    dpi=200,
)
ax.annotate(
    'AC', (-1.75e6, -1.12e6), ha='center', va='top', weight='bold', fontsize=15,
    color=labelcolors['AC'])
ax.annotate(
    'LCC/B2B DC',
    (-1.75e6, -1.24e6), ha='center', va='top', weight='bold', fontsize=15,
    color=labelcolors['DC'])
ax.annotate(
    'VSC DC', (-1.75e6, -1.36e6), ha='center', va='top', weight='bold', fontsize=15,
    color=labelcolors['VSC'])

plt.show()

In [ ]:
### Use modeled transmission routes
routes, wscale, show_overlap = True, 1.5, False
plt.close()
f,ax = rplots.plot_trans_onecase(
    case=os.path.join(runspath,casecomp), pcalabel=False,
    wscale=wscale, routes=routes,
    yearlabel=False, year=year, alpha=0.7,
    show_overlap=show_overlap,
    dpi=200
)
ax.annotate(
    'AC', (-1.75e6, -1.12e6), ha='center', va='top', weight='bold', fontsize=15,
    color=labelcolors['AC'])
ax.annotate(
    'LCC/B2B DC',
    (-1.75e6, -1.24e6), ha='center', va='top', weight='bold', fontsize=15,
    color=labelcolors['DC'])
ax.annotate(
    'VSC DC', (-1.75e6, -1.36e6), ha='center', va='top', weight='bold', fontsize=15,
    color=labelcolors['VSC'])
plt.show()

In [ ]:
### Use modeled transmission routes; only show NEW transmission lines built by 2036
routes, wscale, show_overlap = True, 2.0, False
subtract_baseyear = 2020
plt.close()
f,ax = rplots.plot_trans_onecase(
    case=os.path.join(runspath,casecomp), pcalabel=False,
    wscale=wscale, routes=routes, subtract_baseyear=subtract_baseyear,
    yearlabel=False, year=year, alpha=0.7,
    show_overlap=show_overlap,
)
plt.show()

## VSC lines and converters

In [ ]:
year = 2036
wscale = 0.0004
alpha = 1.0
miles = 300

plt.close()
f,ax = rplots.plot_trans_vsc(
    case=os.path.join(runspath,casecomp),
    year=year, wscale=wscale, alpha=alpha, miles=miles,
)
plt.show()

# Generation capacity maps

In [ ]:
### Plot it
val = 'cap'
plot = 'absdiff'
i_plots = ['wind-ons','upv']
year = 2050


for i_plot in i_plots:
    plt.close()
    f,ax=plt.subplots(
        1,3,sharex=True,sharey=True,figsize=(14,8),
        gridspec_kw={'wspace':-0.05, 'hspace':0.05},
        dpi=150,
    )

    _,_,dfplot = rplots.plotdiffmaps(
        val=val, i_plot=i_plot, year=year,
        casebase=os.path.join(runspath,casebase), casecomp=os.path.join(runspath,casecomp),
        reedspath=reedspath, plot='base', f=f, ax=ax[0], dfba=dfba, dfstates=dfstates,
        cmap=plt.cm.gist_earth_r,
    )
    ax[0].annotate(
        casebase, (0.1,1), xycoords='axes fraction', fontsize=10)

    _,_,dfplot = rplots.plotdiffmaps(
        val=val, i_plot=i_plot, year=year,
        casebase=os.path.join(runspath,casebase), casecomp=os.path.join(runspath,casecomp),
        reedspath=reedspath, plot='comp', f=f, ax=ax[1], dfba=dfba, dfstates=dfstates,
        cmap=plt.cm.gist_earth_r,
    )
    ax[1].annotate(
        casecomp, (0.1,1), xycoords='axes fraction', fontsize=10)

    _,_,dfplot = rplots.plotdiffmaps(
        val=val, i_plot=i_plot, year=year,
        casebase=os.path.join(runspath,casebase), casecomp=os.path.join(runspath,casecomp),
        reedspath=reedspath, plot='absdiff', f=f, ax=ax[2], dfba=dfba, dfstates=dfstates,
        cmap=plt.cm.bwr,
    )
    print(dfplot.CAP_diff.min(), dfplot.CAP_diff.max())
    ax[2].annotate(
        '{}\n– {}'.format(casecomp,casebase), 
        (0.1,1), xycoords='axes fraction', fontsize=10)

    plt.show()

In [ ]:
### Plot it
val = 'cap'
plot = 'absdiff'
i_plots = ['upv','dupv','wind-ons','wind-ofs','battery','nuclear']
cmap = {'upv':plt.cm.Oranges, 'dupv':plt.cm.Reds, 'wind-ons':plt.cm.Blues, 
        'wind-ofs':plt.cm.Purples, 'battery':plt.cm.Greens, 'nuclear':plt.cm.Greys}
year = 2050

for i_plot in i_plots:
    legend_kwds = {'shrink':0.6, 'pad':0, 'orientation':'vertical', 
                   'label':'{} {} {} [GW]'.format(val.upper(), i_plot, year)}
    plt.close()
    f,ax=plt.subplots(figsize=(14,8))

    _,_,dfplot = rplots.plotdiffmaps(
        val=val, i_plot=i_plot, year=year,
        casebase=os.path.join(runspath,casebase), casecomp=os.path.join(runspath,casecomp),
        reedspath=reedspath, plot='comp', f=f, ax=ax, dfba=dfba, dfstates=dfstates,
        cmap=cmap[i_plot], legend_kwds=legend_kwds,
    )
    ax.set_title(casecomp, y=0.98)

    plt.show()

# Process timer

In [ ]:
timecolors = pd.read_csv(
    os.path.join(reedspath,'postprocessing','bokehpivot','in','reeds2','process_style.csv'),
    index_col='order', squeeze=True,
)

In [ ]:
metabase = pd.read_csv(os.path.join(runspath,casebase,'meta.csv'), skiprows=3)

### Subtract time for input_processing script from createmodel.gms
metabase.loc[
    metabase.loc[metabase.process=='createmodel.gms'].index[0],
    'processtime'
] -= (
    metabase.loc[metabase.process.map(lambda x: x.startswith('input_processing')),'processtime'].sum()
    - metabase.loc[metabase.process=='input_processing/calc_financial_inputs.py','processtime'].sum()
)

metabase.loc[
    metabase.loc[metabase.process=='createmodel.gms'].index[0],
    'process'
] = 'createmodel.gms – input_processing/*.py'

dfbase = (metabase.groupby('process', sort=False)['processtime'].sum() / 3600).drop('end')

###### Stacked bars
plt.close()
f,ax = plt.subplots(figsize=(12,1))
dfbase.rename(casebase).to_frame().T.plot.barh(
    ax=ax, stacked=True,
    color=[timecolors.get([c.lower()],'k') for c in dfbase.index]
)
ax.set_xlabel('Run time [hours]')
ax.legend(loc='upper center', bbox_to_anchor=(0.5,-0.7), ncol=3,fontsize=10)
ax.set_xlim(0)
ax.set_yticklabels([])
ax.set_title(casebase)
plots.despine(ax)
plt.show()

###### Bars
df = dfbase.rename(casebase).to_frame().iloc[::-1]
colors = {}
for c in df.index.values:
    if c.startswith('input_processing'):
        colors[c] = 'C0'
    elif 'reeds_augur' in c.lower():
        colors[c] = 'C1'
    elif 'pickle' in c:
        colors[c] = 'C4'
    elif c.startswith('e_') or ('retail' in c):
          colors[c] = 'C2'
    else:
        colors[c] = 'C3'
plt.close()
f,ax = plt.subplots(figsize=(8,7))
ax.barh(
    df.index.values,
    df[casebase].values,
    color=[colors[c] for c in df.index]
)
ax.set_xlabel('Run time [hours]')
ax.set_xlim(0)
ax.set_ylim(-0.5)
plots.despine(ax)
plt.show()

In [ ]:
###### Compare
cases = {'base':casebase, 'comp':casecomp}
meta = {}
for case in cases:
    df = pd.read_csv(os.path.join(runspath,cases[case],'meta.csv'), skiprows=3)

    ### Subtract time for input_processing script from createmodel.gms
    df.loc[
        df.loc[df.process=='createmodel.gms'].index[0],
        'processtime'
    ] -= (
        df.loc[df.process.map(lambda x: x.startswith('input_processing')),'processtime'].sum()
        - df.loc[df.process=='input_processing/calc_financial_inputs.py','processtime'].sum()
    )

    df.loc[
        df.loc[df.process=='createmodel.gms'].index[0],
        'process'
    ] = 'createmodel.gms – input_processing/*.py'

    meta[cases[case]] = df.groupby('process', sort=False)['processtime'].sum() / 3600

dfmeta = pd.concat(meta, axis=1).drop('end')

### Metrics
print('Hours:')
print(dfmeta.sum())
print()
print('Ratio: {:.2f}'.format(dfmeta[casecomp].sum() / dfmeta[casebase].sum()))
print('Difference: {:+.2f} hours'.format(dfmeta[casecomp].sum() - dfmeta[casebase].sum()))

###### Stacked
plt.close()
f,ax = plt.subplots(figsize=(8,2))
dfmeta.T.plot.barh(
    ax=ax, stacked=True,
    color=[timecolors.get([c.lower()],'k') for c in dfbase.index]
)
ax.set_xlabel('Run time [hours]')
ax.legend(loc='upper center', bbox_to_anchor=(0.5,-0.4), ncol=2)
ax.set_xlim(0)
plots.despine(ax)
plt.show()

###### Unstacked
dfplot = dfmeta.iloc[::-1]
colors = {}
for c in dfplot.index.values:
    if c.startswith('input_processing'):
        colors[c] = 'C0'
    elif 'reeds_augur' in c.lower():
        colors[c] = 'C1'
    elif 'pickle' in c:
        colors[c] = 'C4'
    elif c.startswith('e_') or ('retail' in c):
        colors[c] = 'C2'
    else:
        colors[c] = 'C3'
plt.close()
f,ax = plt.subplots(figsize=(7,8))
### Base
ax.barh(
    np.arange(len(dfplot))+0.2,
    dfplot[casebase].values,
    color=[colors[c] for c in dfplot.index],
    height=0.4,
    alpha=1, label=casebase,
)
### Comp
ax.barh(
    np.arange(len(dfplot))-0.2,
    dfplot[casecomp].values,
    color=[colors[c] for c in dfplot.index],
    height=0.4,
    alpha=0.5, label=casecomp,
)
### Formatting
ax.set_xlabel('Run time [hours]')
ax.set_yticks(np.arange(len(dfplot)))
ax.set_yticklabels(dfplot.index.values)
ax.legend()
# ax.set_xlim(0,4)
ax.set_ylim(-0.5)
plots.despine(ax)
plt.show()

# Plot individual site capacity using ReEDS-to-reV

In [ ]:
year = 2036
case = os.path.join(runspath,casecomp)
routes, wscale, show_overlap = True, 1.5, False
subtract_baseyear = None
alpha = 0.25 ### For transmission
colors = 'k' ### For transmission
ms = 1.15
### Example using a different CRS
# crs = (
#     '+proj=aea +lat_0=37.5 +lon_0=-96 +lat_1=29.5 +lat_2=45.5 '
#     '+x_0=0 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'
# )
### reV CRS
crs = 'ESRI:102008'

plt.close()
f,ax = rplots.plot_vresites_transmission(
    case, year, crs=crs,
    routes=routes, wscale=wscale, show_overlap=show_overlap,
    subtract_baseyear=subtract_baseyear,
    alpha=alpha, colors=colors, ms=ms,
)
plt.show()